In [2]:
import re
import os
import sys
import json
import langchain
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.graph import START, StateGraph, MessagesState
from langchain_core.messages import SystemMessage, HumanMessage
from schemas.mondial_federated_schema import MONDIAL_ECONOMY, MONDIAL_GEO, MONDIAL_POlITICS, MONDIAL_SOCIAL, MONDIAL_FULL_SCHEMA

load_dotenv()

True

In [2]:
# Leitura do arquivo PDF extraído como texto
with open("schemas/mondial-RS.txt", "r") as file:
    content = file.read()

table_chunks = re.findall(r"---\n(.*?)\n---", content, flags=re.DOTALL)
table_chunks

['Economy: economical information about the countries.\ncountry: the country code\nGDP: gross domestic product (in million $)\nagriculture: percentage of agriculture of the GDP\nservice: percentage of services of the GDP\nindustry: percentage of industry of the GDP\ninflation: inflation rate (per annum)\nunemployment: unemployment rate',
 'Population: information about the population of the countries.\ncountry: the country code\npopulation growth: population growth rate (per annum)\ninfant mortality: infant mortality (per thousand)',
 'CountryLocalName: information about the local name of the country.\ncountry: the country code\nlocalname: the local name, usually in a local alphabet (UTF-8)',
 'Religion: information about the religions in a country\ncountry: the country code\nname: name of the religion\npercentage: percentage of the language in this country',
 'borders: informations about neighboring countries. Note that in this relation, for every pair of\nneighboring countries (A,B),

In [3]:
import re
import json

# Função para processar cada chunk e extrair informações
def process_chunks(table_chunks):
    tables = {}

    for chunk in table_chunks:
        # Separar o nome da tabela e descrição
        header_match = re.match(r"^(.*?): (.*?)\n", chunk, flags=re.DOTALL)
        if not header_match:
            continue
        table_name = header_match.group(1).strip()
        description = header_match.group(2).strip()
        
        # Extrair colunas e descrições
        columns = {}
        for line in chunk.split("\n")[1:]:  # Ignorar a primeira linha
            column_match = re.match(r"^(.*?): (.*)", line)
            if column_match:
                column_name = column_match.group(1).strip()
                column_description = column_match.group(2).strip()
                columns[column_name] = column_description
        
        # Estruturar os dados da tabela
        tables[table_name] = {
            "description": description,
            "columns": columns
        }
    
    return tables

# Processar os chunks
structured_data = process_chunks(table_chunks)

# Exibir o resultado
print(json.dumps(structured_data, indent=2, ensure_ascii=False))

{
  "Economy": {
    "description": "economical information about the countries.",
    "columns": {
      "country": "the country code",
      "GDP": "gross domestic product (in million $)",
      "agriculture": "percentage of agriculture of the GDP",
      "service": "percentage of services of the GDP",
      "industry": "percentage of industry of the GDP",
      "inflation": "inflation rate (per annum)",
      "unemployment": "unemployment rate"
    }
  },
  "Population": {
    "description": "information about the population of the countries.",
    "columns": {
      "country": "the country code",
      "population growth": "population growth rate (per annum)",
      "infant mortality": "infant mortality (per thousand)"
    }
  },
  "CountryLocalName": {
    "description": "information about the local name of the country.",
    "columns": {
      "country": "the country code",
      "localname": "the local name, usually in a local alphabet (UTF-8)"
    }
  },
  "Religion": {
    "de

In [4]:
from schemas.mondial_federated_schema import MONDIAL_ECONOMY, MONDIAL_GEO, MONDIAL_POlITICS, MONDIAL_SOCIAL, MONDIAL_FULL_SCHEMA

economy_db_tables_names = []
economy_db_tables = MONDIAL_ECONOMY.split("CREATE TABLE ")
for table in economy_db_tables:
    table_name = table.split("\n")[0]
    economy_db_tables_names.append(table_name)
    
economy_db_tables_names = economy_db_tables_names[1:]
print(economy_db_tables_names)

geo_db_tables_names = []
geo_db_tables = MONDIAL_GEO.split("CREATE TABLE ")
for table in geo_db_tables:
    table_name = table.split("\n")[0]
    geo_db_tables_names.append(table_name)

geo_db_tables_names = geo_db_tables_names[1:]
print(geo_db_tables_names)

social_db_tables_names = []
social_db_tables = MONDIAL_SOCIAL.split("CREATE TABLE ")
for table in social_db_tables:
    table_name = table.split("\n")[0]
    social_db_tables_names.append(table_name)

social_db_tables_names = social_db_tables_names[1:]
print(social_db_tables_names)

politics_db_tables_names = []
politics_db_tables = MONDIAL_POlITICS.split("CREATE TABLE ")
for table in politics_db_tables:
    table_name = table.split("\n")[0]
    politics_db_tables_names.append(table_name)
    
politics_db_tables_names = politics_db_tables_names[1:]
print(politics_db_tables_names)

['Economy', 'Population']
['Country', 'RiverThrough', 'encompasses', 'Continent', 'City', 'Province', 'Mountain', 'Desert', 'Island', 'Lake', 'Sea', 'River', 'Airport', 'geo_Mountain', 'geo_Desert', 'geo_Island', 'geo_River', 'geo_Sea', 'geo_Lake', 'geo_Source', 'geo_Estuary', 'located', 'locatedOn', 'islandIn', 'MountainOnIsland', 'LakeOnIsland', 'RiverOnIsland', 'mergesWith']
['Religion', 'EthnicGroup', 'Language', 'Sublanguage', 'Countrylocalname', 'Countryothername', 'Provincelocalname', 'Provinceothername', 'Citylocalname', 'Cityothername', 'Countrypops', 'Provpops', 'Citypops']
['Politics', 'borders', 'Organization', 'isMember']


In [11]:
prompt = """
You are an intelligent assistant responsible for identifying `SameAsTable` relationships between tables in different federated databases. Your task is to determine columns in the input table that represent equivalent data or have the same logical meaning as columns in tables from other databases.

!!! Attention !!!
1. Respond **only in JSON format**, with no additional text or explanation.
2. Identify **only SameAsTable relationships**.
3. Prioritize connections between the input table and tables in other databases. Avoid connections within the same database.

Example:
The `Economy` table contains economic information. A relevant `SameAsTable` connection might be:
{{
    "Endpoint Source": "MONDIAL_ECONOMY",
    "Class Source": "Economy",
    "Properties Source": "country",
    "Endpoint Destination": "MONDIAL_POLITICS",
    "Class Destination": "Politics",
    "Properties Destination": "country"
}}

Now, for the table `{table_name}`:
Description: {table_description}
Columns and descriptions:
{table_columns_info}

Schemas of federated databases:
MONDIAL_SOCIAL DATABASE SCHEMA:
{social_db_schema}

MONDIAL_GEO DATABASE SCHEMA:
{geo_db_schema}

MONDIAL_POLITICS DATABASE SCHEMA:
{politics_db_schema}

MONDIAL_ECONOMY DATABASE SCHEMA:
{economy_db_schema}

Question:
Identify all `SameAsTable` relationships for the table `{table_name}`. Return the relationships **only in JSON format**.
"""

In [12]:
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o", temperature=0)

for table_name in structured_data:
    table_info = structured_data[table_name]
    
    table_description = table_info["description"]
    table_columns_info = "\n".join([f"{column}: {description}" for column, description in table_info["columns"].items()])
    
    social_db_schema = MONDIAL_SOCIAL
    geo_db_schema = MONDIAL_GEO
    politics_db_schema = MONDIAL_POlITICS
    economy_db_schema = MONDIAL_ECONOMY
    
    prompt_with_schema = prompt.format(
        table_name=table_name,
        table_description=table_description,
        table_columns_info=table_columns_info,
        social_db_schema=social_db_schema,
        geo_db_schema=geo_db_schema,
        politics_db_schema=politics_db_schema,
        economy_db_schema=economy_db_schema
    )
    
    # print(prompt_with_schema)
    
    human_msg = HumanMessage(
        content=prompt_with_schema
    )
    
    result = llm | StrOutputParser()
    
    print(result.invoke([human_msg]))
    
    break

In [25]:
import json

# Exemplo de resposta da LLM
responses = [
    {   
        "Table": "Economy",
        "Description": "Economic information about the countries.",
        "Database": "MONDIAL_ECONOMY",
        "SameAsTable": {
            "Endpoint Source": "MONDIAL_ECONOMY",
            "Class Source": "Economy",
            "Properties Source": "country",
            "Endpoint Destination": "MONDIAL_GEO",
            "Class Destination": "Country",
            "Properties Destination": "Code"
        },
        "ExternalObjectProperty": [
            {
                "Domain Endpoint": "MONDIAL_ECONOMY",
                "Domain Class": "Economy",
                "Domain Object Property": "country",
                "Range Endpoint": "MONDIAL_SOCIAL",
                "Range Class": "Religion",
                "Range Object Property": "Country"
            },
            {
                "Domain Endpoint": "MONDIAL_ECONOMY",
                "Domain Class": "Economy",
                "Domain Object Property": "country",
                "Range Endpoint": "MONDIAL_SOCIAL",
                "Range Class": "EthnicGroup",
                "Range Object Property": "Country"
            },
            {
                "Domain Endpoint": "MONDIAL_ECONOMY",
                "Domain Class": "Economy",
                "Domain Object Property": "country",
                "Range Endpoint": "MONDIAL_SOCIAL",
                "Range Class": "Language",
                "Range Object Property": "Country"
            },
            {
                "Domain Endpoint": "MONDIAL_ECONOMY",
                "Domain Class": "Economy",
                "Domain Object Property": "country",
                "Range Endpoint": "MONDIAL_GEO",
                "Range Class": "Country",
                "Range Object Property": "Code"
            },
            {
                "Domain Endpoint": "MONDIAL_ECONOMY",
                "Domain Class": "Economy",
                "Domain Object Property": "country",
                "Range Endpoint": "MONDIAL_POLITICS",
                "Range Class": "Politics",
                "Range Object Property": "Country"
            }
        ]
    }
    # Adicione mais respostas aqui
]

# Consolidar as respostas
consolidated_data = {
    "SameAsTable": [],
    "ExternalObjectProperty": []
}

for response in responses:
    # Adicionar SameAsTable
    if "SameAsTable" in response:
        consolidated_data["SameAsTable"].append(response["SameAsTable"])
    
    # Adicionar ExternalObjectProperty
    if "ExternalObjectProperty" in response:
        consolidated_data["ExternalObjectProperty"].extend(response["ExternalObjectProperty"])

# Salvar em arquivo JSON
with open("consolidated_relationships.json", "w") as json_file:
    json.dump(consolidated_data, json_file, indent=2, ensure_ascii=False)

# Opcional: Salvar em YAML
import yaml

with open("consolidated_relationships.yaml", "w") as yaml_file:
    yaml.dump(consolidated_data, yaml_file, allow_unicode=True, default_flow_style=False)

print("Consolidação concluída! Arquivos salvos.")

In [26]:
consolidated_data["SameAsTable"]

In [27]:
consolidated_data["ExternalObjectProperty"]

In [46]:
subquery_prompt = """
You are an intelligent assistant specialized in federated databases. Your task is to analyze a query and determine how it can be split into subqueries for different federated databases. 

Here is the schema of the federated database:
1. MONDIAL_ECONOMY:
{economy_schema}
2. MONDIAL_SOCIAL:
{social_schema}
3. MONDIAL_GEO:
{geo_schema}
4. MONDIAL_POLITICS:
{politics_schema}

Here are the relationships between tables across schemas:
1. SameAsTable relationships:
{same_as_table}
2. ExternalObjectProperty relationships:
{external_object_property}

### Query
"{query}"

### Task
1. Identify the tables and columns relevant to this query.
2. Indicate which federated schema each table belongs to.
3. Suggest SQL subqueries for each schema to retrieve the required data.
4. Specify conditions that can be applied within each subquery.

Answer ALL AND ONLY IN JSON format with the following structure:
{{
  "subqueries": [
    {{
      "schema": "Schema Name",
      "tables": ["Table1", "Table2"],
      "columns": ["Column1", "Column2"],
      "conditions": ["Condition1", "Condition2"],
      "sql": "Generated SQL"
    }}
  ]
}}
"""

integration_prompt = """
You are an intelligent assistant responsible for integrating results from multiple federated databases.

Here are the SQL subqueries already generated:
{subqueries}

Here are the relationships between tables:
1. SameAsTable relationships:
{same_as_table}
2. ExternalObjectProperty relationships:
{external_object_property}

Task:
1. Identify how the results of these subqueries should be combined (JOIN or UNION).
2. Specify the columns used for integration (e.g., common keys).
3. Generate the final SQL query to integrate all subqueries.

Answer ALL AND ONLY IN JSON format:
{{
  "integration_plan": {{
    "method": "JOIN/UNION",
    "columns": ["Column1", "Column2"],
    "sql": "Generated SQL"
  }}
}}
"""

In [47]:
from schemas.mondial_federated_schema import MONDIAL_ECONOMY, MONDIAL_GEO, MONDIAL_POlITICS, MONDIAL_SOCIAL, MONDIAL_FULL_SCHEMA

def get_subqueries(query):
    prompt = subquery_prompt.format(
        economy_schema=MONDIAL_ECONOMY,
        social_schema=MONDIAL_SOCIAL,
        geo_schema=MONDIAL_GEO,
        politics_schema=MONDIAL_POlITICS,
        same_as_table=json.dumps(consolidated_data["SameAsTable"]),
        external_object_property=json.dumps(consolidated_data["ExternalObjectProperty"]),
        query=query
    )
    
    human_msg = HumanMessage(
        content=prompt
    )
    
    chain = llm | StrOutputParser()
    result = chain.invoke([human_msg])
    
    return result

print(get_subqueries("Retrieve the GDP and religion of countries in South America."))

```json
{
  "subqueries": [
    {
      "schema": "MONDIAL_GEO",
      "tables": ["Country", "encompasses"],
      "columns": ["Code", "Name"],
      "conditions": ["encompasses.Continent = 'South America'", "Country.Code = encompasses.Country"],
      "sql": "SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America'"
    },
    {
      "schema": "MONDIAL_ECONOMY",
      "tables": ["Economy"],
      "columns": ["Country", "GDP"],
      "conditions": ["Economy.Country IN (SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America')"],
      "sql": "SELECT Economy.Country, Economy.GDP FROM Economy WHERE Economy.Country IN (SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America')"
    },
    {
      "schema": "MONDIAL_SOCIAL",
      "tables": ["Religion"],
      "c

In [48]:
subqueries = """
{
  "subqueries": [
    {
      "schema": "MONDIAL_GEO",
      "tables": ["Country", "encompasses"],
      "columns": ["Code", "Name"],
      "conditions": ["encompasses.Continent = 'South America'", "Country.Code = encompasses.Country"],
      "sql": "SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America'"
    },
    {
      "schema": "MONDIAL_ECONOMY",
      "tables": ["Economy"],
      "columns": ["Country", "GDP"],
      "conditions": ["Economy.Country IN (SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America')"],
      "sql": "SELECT Economy.Country, Economy.GDP FROM Economy WHERE Economy.Country IN (SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America')"
    },
    {
      "schema": "MONDIAL_SOCIAL",
      "tables": ["Religion"],
      "columns": ["Country", "Name", "Percentage"],
      "conditions": ["Religion.Country IN (SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America')"],
      "sql": "SELECT Religion.Country, Religion.Name, Religion.Percentage FROM Religion WHERE Religion.Country IN (SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America')"
    }
  ]
}"""

def join_subqueries(subqueries):
    prompt = integration_prompt.format(
        subqueries=subqueries,
        same_as_table=json.dumps(consolidated_data["SameAsTable"]),
        external_object_property=json.dumps(consolidated_data["ExternalObjectProperty"])
    )
    
    human_msg = HumanMessage(
        content=prompt
    )
    
    chain = llm | StrOutputParser()
    result = chain.invoke([human_msg])
    
    return result

print(join_subqueries(subqueries))

```json
{
  "integration_plan": {
    "method": "JOIN",
    "columns": ["Country"],
    "sql": "SELECT e.Country, e.GDP, r.Name AS Religion, r.Percentage FROM (SELECT Economy.Country, Economy.GDP FROM Economy WHERE Economy.Country IN (SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America')) AS e JOIN (SELECT Religion.Country, Religion.Name, Religion.Percentage FROM Religion WHERE Religion.Country IN (SELECT Country.Code FROM Country JOIN encompasses ON Country.Code = encompasses.Country WHERE encompasses.Continent = 'South America')) AS r ON e.Country = r.Country"
  }
}
```


In [1]:
from langchain_utils import get_llm

llm = get_llm()
llm.invoke("What is the GDP and religion of countries in South America?")

AIMessage(content='South America is a diverse continent with various countries, each having its own economic and religious landscape. Here is a general overview of the GDP and predominant religions in some South American countries:\n\n1. **Argentina**\n   - **GDP (Nominal, 2021):** Approximately $491 billion USD\n   - **Religion:** Predominantly Roman Catholic, with a significant number of people identifying as non-religious or atheist, and a growing number of Protestant Christians.\n\n2. **Bolivia**\n   - **GDP (Nominal, 2021):** Approximately $36 billion USD\n   - **Religion:** Predominantly Roman Catholic, with a significant Protestant Christian minority and indigenous spiritual practices.\n\n3. **Brazil**\n   - **GDP (Nominal, 2021):** Approximately $1.61 trillion USD\n   - **Religion:** Predominantly Roman Catholic, with a large and growing number of Protestant Christians, as well as Afro-Brazilian religions and increasing secularism.\n\n4. **Chile**\n   - **GDP (Nominal, 2021):**